In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
#importando datasets http://www.respeitoavida.sp.gov.br/relatorios/
df_obitos_raw=pd.read_csv('obitos_publico.csv',encoding='unicode_escape',sep=';')
df_nao_fatais_raw = pd.read_csv('acidentes_naofatais.csv',encoding='unicode_escape',sep=';')
df_fatais_raw = pd.read_excel('acidentes_fatais.xlsx')
# df_fatais_rod_raw = pd.read_excel('acidentes_fatais_rodovias.xlsx') # Este df já está incluso no de cima

C:\Users\Renato\anaconda3\envs\Data_Science\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
#checando duplicatas
df_obitos_raw.duplicated().value_counts() #achado 97 linhas duplicadas (os outros df's não tem duplicatas)
df_obitos_raw = df_obitos_raw.drop_duplicates()#dropando as 97 duplicatas, temos 34657 linhas

In [4]:
#SABENDO QUE TEMOS 3 DATAFRAMES, A IDEIA É JUNTAR TODOS E TER UMA COLUNA QUE CLASSIFICA SE O ACIDENTE FOI LETAL OU NÃO.

In [279]:
# Criando Dataframes para trabalhar
df_nao_fatais = df_nao_fatais_raw.copy()

df_fatais = df_fatais_raw.copy()#NAO TEMOS COMO FILTRAR POR MOTOCILCETA POIS NAO TEM ESSA COLUNA!!!!

df_obitos = df_obitos_raw.copy()#AQUI CONSEGUIMOS FILTRAR POR MOTOCICLETA

In [280]:
df_fatais.columns

Index(['ID', 'Id Delegacia (RDO)', 'Número BO (RDO)', 'Ano BO (RDO)',
       'Data do Acidente', 'Dia do Acidente', 'Mês do Acidente',
       'Ano do Acidente', 'Ano/Mês do Acidente', 'Dia da semana',
       'Hora do Acidente', 'Turno', 'Município', 'Região Administrativa',
       'Logradouro', 'Numeral / KM', 'Jurisdição', 'Administração',
       'Conservação', 'Tipo do Local do Acidente', 'Lat (GEO)', 'Long (GEO)',
       'Iluminação da via (SIOPM)', 'Superfície da Via (SIOPM)',
       'Tipo de pista (SIOPM)', 'Outro Veículo Envolvido', 'Tipo de via',
       'Condições Climáticas (SIOPM)', 'Sentido da Via (SIOPM)',
       'Limite da velocidade da via (SIOPM)', 'Quantidade de vítimas',
       'Tempo entre o Acidente e as Mortes'],
      dtype='object')

In [281]:
df_fatais['Tipo de pista (SIOPM)'].unique()

array(['NAO DISPONIVEL', 'DUPLA', 'SIMPLES', 'MULTIPLA'], dtype=object)

In [282]:
df_nao_fatais.columns

Index(['ID', 'Data do Acidente', 'Dia do Acidente', 'Mês do Acidente',
       'Ano do Acidente', 'Ano/Mês do Acidente', 'Dia da Semana',
       'Hora do Acidente', 'Turno', 'Município', 'Região Administrativa',
       'Logradouro', 'Numero/KM', 'Jurisdição', 'Administração', 'Conservação',
       'LAT_(GEO)', 'LONG_(GEO)', 'Condições da pista', 'Condições Climáticas',
       'Iluminação', 'Mão de direção', 'Obras na pista', 'Relevo',
       'Superfície da via', 'Tipo de pavimento', 'Tipo de pista', 'Traçado',
       'Veículos Envolvidos - Bicicleta', 'Veículos Envolvidos - Caminhão',
       'Veículos Envolvidos - Automóvel', 'Veículos Envolvidos - Outros',
       'Veículos Envolvidos - Motocicleta', 'Veículos Envolvidos - Ônibus',
       'Veículos Envolvidos - ND', 'Veículos Envolvidos - Pedestre',
       'Pessoas Envolvidas - Grave', 'Pessoas Envolvidas - Ileso',
       'Pessoas Envolvidas - Leve', 'Pessoas Envolvidas - Fatal',
       'Pessoas Envolvidas - Feridos', 'Pessoas Envolvida

In [283]:
df_nao_fatais['Condições da pista'].unique()

array(['NAO DISPONIVEL', 'MÚLTIPLA', 'DUPLA', 'SIMPLES', 'MULTIPLA',
       'PLANO', 'INCLINADO'], dtype=object)

In [284]:
# NAO FATAIS
#tratando as coordenadas geográficas
df_nao_fatais = df_nao_fatais[df_nao_fatais['LAT_(GEO)']!='NAO DISPONIVEL']  #dropando mais de 35k linhas sem latitude e longitude de 377k, DISCUTIR ISSO!
df_nao_fatais = df_nao_fatais[df_nao_fatais['LONG_(GEO)']!='NAO DISPONIVEL'] 
df_nao_fatais['LAT_(GEO)'] = df_nao_fatais['LAT_(GEO)'].str.replace(',','.') #susbstituindo ',' por '.' 
df_nao_fatais['LAT_(GEO)'] = df_nao_fatais['LAT_(GEO)'].astype(float)#transformando pra float
df_nao_fatais['LONG_(GEO)'] = df_nao_fatais['LONG_(GEO)'].str.replace(',','.') #susbstituindo ',' por '.'
df_nao_fatais['LONG_(GEO)'] = df_nao_fatais['LONG_(GEO)'].astype(float)#transformando pra float

#alterando nomes de colunas
df_nao_fatais.rename(columns={'LAT_(GEO)':'LATITUDE','LONG_(GEO)':'LONGITUDE','Condições da pista':'Tipo de pista (SIOPM)'}, inplace=True)

#alterando tipos de variáveis
df_nao_fatais['Data do Acidente'] = pd.to_datetime(df_nao_fatais['Data do Acidente'],format='%Y-%m-%d')#,errors='coerce'
df_nao_fatais['Ano do Acidente'] = pd.to_datetime(df_nao_fatais['Ano do Acidente'],format='%Y')
df_nao_fatais['Dia do Acidente'] = pd.to_datetime(df_nao_fatais['Dia do Acidente'],format='%d')


In [286]:
# Filtros: Motocicleta, Metropolitana São Paulo, Entre 2019 e 2020
df_nao_fatais_sp_moto = df_nao_fatais.loc[df_nao_fatais['Veículos Envolvidos - Motocicleta']==1] #de 377k, 153k é moto
df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Região Administrativa']=='Metropolitana de São Paulo']# dos 153k, 54692 é SP
df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Data do Acidente']>='2019-01-01']
df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.loc[df_nao_fatais_sp_moto['Data do Acidente']<='2020-12-31']

In [287]:
#FATAIS
#tratando as coordenadas geográficas
df_fatais = df_fatais[df_fatais['Lat (GEO)']!='NAO DISPONIVEL'] #dropando mais de 3k linhas sem latitude e longitude, DISCUTIR ISSO!
df_fatais = df_fatais[df_fatais['Long (GEO)']!='NAO DISPONIVEL']
df_fatais['Lat (GEO)'] = df_fatais['Lat (GEO)'].str.replace(',','.') #susbstituindo ',' por '.' 
df_fatais['Lat (GEO)'] = df_fatais['Lat (GEO)'].astype(float)#transformando pra float
df_fatais['Long (GEO)'] = df_fatais['Long (GEO)'].str.replace(',','.') #susbstituindo ',' por '.' 
df_fatais['Long (GEO)'] = df_fatais['Long (GEO)'].astype(float)#transformando pra float

#alterando nomes de colunas
df_fatais.rename(columns={'Lat (GEO)':'LATITUDE','Long (GEO)':'LONGITUDE','Id Delegacia (RDO)':'ID Delegacia','Número BO (RDO)':'Número BO','Ano BO (RDO)':'Ano BO','Tipo de via':'Tipo de Via','Dia da semana':'Dia da Semana','Condições Climáticas (SIOPM)':'Condições Climáticas'}, inplace=True)

#alterando tipos de variáveis
df_fatais.loc[31790,'Data do Acidente']='2020-02-27'
df_fatais['Data do Acidente'] = pd.to_datetime(df_fatais['Data do Acidente'],format='%Y-%m-%d')#,errors='coerce'
df_fatais.loc[31790, 'Ano do Acidente']='2020'
df_fatais['Ano do Acidente'] = pd.to_datetime(df_fatais['Ano do Acidente'],format='%Y')
df_fatais['Dia do Acidente'] = pd.to_datetime(df_fatais['Dia do Acidente'],format='%d')

In [288]:
# Filtros: Metropolitana São Paulo, Entre 2019 e 2020
df_fatais_sp = df_fatais.loc[df_fatais['Região Administrativa']=='Metropolitana de São Paulo']
df_fatais_sp = df_fatais_sp.loc[df_fatais_sp['Data do Acidente']>='2019-01-01']#dos 9k na regiado de SP, ficamos com 2,7k >=2019
df_fatais_sp = df_fatais_sp.loc[df_fatais_sp['Data do Acidente']<='2020-12-31']# parece que nao tem dados >31-dez-2020, estranho

In [289]:
#ÓBITOS PUBLICO
#tratando as coordenadas geográficas
df_obitos = df_obitos[df_obitos['LAT_(GEO)']!='NAO DISPONIVEL'] #dropando mais de 3k linhas sem latitude e longitude, DISCUTIR ISSO!
df_obitos = df_obitos[df_obitos['LONG_(GEO)']!='NAO DISPONIVEL']
df_obitos['LAT_(GEO)'] = df_obitos['LAT_(GEO)'].str.replace(',','.') #susbstituindo ',' por '.' 
df_obitos['LAT_(GEO)'] = df_obitos['LAT_(GEO)'].astype(float)#transformando pra float
df_obitos['LONG_(GEO)'] = df_obitos['LONG_(GEO)'].str.replace(',','.') #susbstituindo ',' por '.' 
df_obitos['LONG_(GEO)'] = df_obitos['LONG_(GEO)'].astype(float)#transformando pra float

#alterando nomes de colunas
df_obitos.rename(columns={'LAT_(GEO)':'LATITUDE','LONG_(GEO)':'LONGITUDE','Id da Delegacia (RDO)':'ID Delegacia','Número do Bo (RDO)':'Número BO','Ano do BO (RDO)':'Ano BO','Tipo de via':'Tipo de Via'}, inplace=True)

#alterando tipos de variáveis
df_obitos['Data do Acidente'] = pd.to_datetime(df_obitos['Data do Acidente'],errors='coerce')

In [290]:
# Filtrando região metropolitana de SP
df_obitos_sp = df_obitos.loc[df_obitos['Região Administrativa']=='Metropolitana de São Paulo']
df_obitos_sp = df_obitos_sp.loc[df_obitos_sp['Data do Acidente']>='2019-01-01']
df_obitos_sp = df_obitos_sp.loc[df_obitos_sp['Data do Acidente']<='2020-12-31']

In [291]:
#objetivo é saber quais acidentes fatais foram de motociclistas!!

In [292]:
df_fatais_sp_merged = df_fatais_sp.merge(df_obitos_sp,
left_on=['Número BO','Data do Acidente','LATITUDE','ID Delegacia','Ano BO','Hora do Acidente','Turno','Município','Região Administrativa','LONGITUDE','Jurisdição','Administração','Conservação','Outro Veículo Envolvido','Tipo de Via','Logradouro','Dia da Semana'],
right_on=['Número BO','Data do Acidente','LATITUDE','ID Delegacia','Ano BO','Hora do Acidente','Turno','Município','Região Administrativa','LONGITUDE','Jurisdição','Administração','Conservação','Outro Veículo Envolvido','Tipo de Via','Logradouro','Dia da Semana']) 
#com as 3 primeiras condições temos um novo df do mesmo tamanho do df obitos...porém coloquei mais condições para as colunas nao sairem duplicadas

In [293]:
df_fatais_sp_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2888 entries, 0 to 2887
Data columns (total 49 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   ID                                   2888 non-null   int64         
 1   ID Delegacia                         2888 non-null   int64         
 2   Número BO                            2888 non-null   int64         
 3   Ano BO                               2888 non-null   int64         
 4   Data do Acidente                     2888 non-null   datetime64[ns]
 5   Dia do Acidente                      2888 non-null   datetime64[ns]
 6   Mês do Acidente                      2888 non-null   object        
 7   Ano do Acidente                      2888 non-null   datetime64[ns]
 8   Ano/Mês do Acidente                  2888 non-null   float64       
 9   Dia da Semana                        2888 non-null   object        
 10  Hora do Acid

In [294]:
#FILTRANDO O NOVO DF POR MOTOCICLETA
df_fatais_sp_moto_merged = df_fatais_sp_merged[df_fatais_sp_merged['Tipo do veículo da vítima']=='MOTOCICLETA']

In [296]:
#agora precisamos dropar as colunas que não iremos usar na junção dos DF's fatais e não fatais.
df_nao_fatais_sp_moto = df_nao_fatais_sp_moto.drop(columns=
['ID','Ano/Mês do Acidente', 
        'Turno', 'Região Administrativa',
       'Logradouro', 'Numero/KM', 'Jurisdição', 'Administração', 'Conservação',
       'Iluminação', 'Mão de direção', 'Obras na pista', 'Relevo',
       'Superfície da via', 'Tipo de pavimento', 'Traçado','Tipo de pista',
       'Veículos Envolvidos - Bicicleta', 'Veículos Envolvidos - Caminhão',
       'Veículos Envolvidos - Automóvel', 'Veículos Envolvidos - Outros',
       'Veículos Envolvidos - Motocicleta', 'Veículos Envolvidos - Ônibus',
       'Veículos Envolvidos - ND', 'Veículos Envolvidos - Pedestre',
       'Pessoas Envolvidas - Grave', 'Pessoas Envolvidas - Ileso',
       'Pessoas Envolvidas - Leve', 'Pessoas Envolvidas - Fatal',
       'Pessoas Envolvidas - Feridos', 'Pessoas Envolvidas - ND',
       'Serviço de Atendimento - Bombeiro', 'Serviço de Atendimento - PMRV',
       'Serviço de Atendimento - PRF',
       'Serviço de Atendimento - Radio Patrulha',
       'Servciço de Atendimento - CPTRAN',
       'Tipo de Acidente - Atropelamento', 'Tipo de Acidente - Choque',
       'Tipo de Acidente - Colisão',
       'Tipo de Acidente - Outros tipos de Acidente', 'Tipo de Acidente - ND',
       'Turno + Dia de Semana', 'Tempo entre o Acidente e as Mortes'])

In [299]:
df_fatais_sp_moto_merged = df_fatais_sp_moto_merged.drop(columns=[
    'ID', 'ID Delegacia', 'Número BO', 'Ano BO',
       'Ano/Mês do Acidente', 'Turno',
       'Região Administrativa', 'Logradouro', 'Numeral / KM',
       'Jurisdição', 'Administração', 'Conservação',
       'Tipo do Local do Acidente',
       'Iluminação da via (SIOPM)', 'Superfície da Via (SIOPM)', 'Outro Veículo Envolvido', 'Sentido da Via (SIOPM)',
       'Limite da velocidade da via (SIOPM)', 'Quantidade de vítimas',
       'Tempo entre o Acidente e as Mortes', 'Data do Óbito', 'Dia do óbito',
       'Mês do Óbito', 'Mês Óbito Descrição', 'Ano do Óbito',
       'Ano/Mês do Óbito', 'Númeral / KM',
       'Tipo do local da ocorrência', 'Tipo do veículo da vítima',
       'Tipo de vítima', 'Local do óbito', 'Tipo de acidente',
       'Sub Tipo do Acidente', 'Sexo', 'Faixa etária', 'Idade da vítima',
       'Tempo entre o Acidente e o Óbito'])

In [313]:
#criando o label fatal e não fatal e concatenando
df_fatais_sp_moto_merged['Acidente fatal']=1
df_fatais_sp_moto_merged['Acidente não fatal']=0
df_nao_fatais_sp_moto['Acidente fatal']=0
df_nao_fatais_sp_moto['Acidente não fatal']=1

#concat
df_acidentes_sp_moto = pd.concat([df_nao_fatais_sp_moto,df_fatais_sp_moto_merged])

In [314]:
df_acidentes_sp_moto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52130 entries, 27347 to 2887
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Data do Acidente       52130 non-null  datetime64[ns]
 1   Dia do Acidente        52130 non-null  datetime64[ns]
 2   Mês do Acidente        52130 non-null  object        
 3   Ano do Acidente        52130 non-null  datetime64[ns]
 4   Dia da Semana          52130 non-null  object        
 5   Hora do Acidente       52130 non-null  object        
 6   Município              52130 non-null  object        
 7   LATITUDE               52130 non-null  float64       
 8   LONGITUDE              52130 non-null  float64       
 9   Tipo de pista (SIOPM)  52130 non-null  object        
 10  Condições Climáticas   52130 non-null  object        
 11  Tipo de Via            52130 non-null  object        
 12  Acidente fatal         52130 non-null  int64         
 13

In [316]:
# export
df_acidentes_sp_moto.to_csv('df_acidentes_sp_moto.csv')